In [1]:
import pandas as pd
import numpy as np
import os
import glob
import random
from tqdm import tqdm_notebook as tqdm

from support import *
from features.feature_ts import genX
from experiment.algorithms.cluster_prep import *
from Gauss_fit_functions import extractFIT, extractToPs 

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\os.py
c:\Users\Jason\thesis_project


# Get Gauss-Fit Features

### Use a small subgroup from the dataset to create feature extraction code
##### The profiles chosen must:
##### - Be typical residential users
##### - Have enough data collected for winter/weekdays
##### - Have a gauss fit with a low error

In [2]:
X = genX([1994,2014],drop_0 = True)

dropping all zero rows


In [3]:
# Collect only winter weekday profiles from dataset

X.reset_index(inplace = True)

df = X.copy()

# Extract Season
df['month'] = df.date.dt.month
df['season'] = df['month'].apply(lambda x: 'winter' if x in [6, 7, 8] else 'summer') 
df_winter = df[df['season'] == 'winter'] # Create dataframe with all the winter months, excluding weekends


# Extract Weekdays
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']
df_winter['day_names'] = df_winter.date.dt.day_name()
df_winter['daytype'] = df_winter.day_names.where(~df_winter.day_names.isin(weekdays), 'weekday')
df_winter.drop(['day_names'], axis = 1, inplace = True)
df_winter_weekdays  = df_winter[df_winter['daytype'] == 'weekday'] # Create dataframe with only weekdays
df_winter_weekdays.drop(['month', 'season','daytype'], axis = 1, inplace = True)
df_winter_weekdays.set_index(['ProfileID','date'], inplace=True)

<ipython-input-3-e37b2df3bdee>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-3-e37b2df3bdee>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [4]:
df_winter_weekdays

0         1         2         3         4  \
ProfileID date                                                           
1         1994-06-20  1.058333  1.625000  0.341667  2.100000  1.225000   
          1994-06-21  0.425000  2.350000  0.550000  0.291667  2.100000   
          1994-06-22  0.741667  2.325000  0.583333  1.191667  1.225000   
          1994-06-23  1.166667  1.883333  0.450000  0.541667  2.333333   
          1994-06-24  2.408333  0.591667  0.316667  2.341667  0.550000   
...                        ...       ...       ...       ...       ...   
12029071  2014-08-25  0.246667  0.333333  0.253333  0.206667  0.300000   
          2014-08-26  0.273333  0.273333  0.193333  0.186667  0.266667   
          2014-08-27  0.320000  0.286667  0.260000  0.240000  0.220000   
          2014-08-28  0.266667  0.266667  0.366667  0.286667  0.240000   
          2014-08-29  0.273333  0.273333  0.346667  0.300000  0.273333   

                             5         6         7         8         9  ...  \
ProfileID date                                                          ...   
1         1994-06-20  1.058333  4.808333  4.091667  2.083333  0.475000  ...   
          1994-06-21  0.591667  4.066667  2.791667  0.816667  2.100000  ...   
          1994-06-22  0.575000  4.066667  3.350000  0.866667  2.808333  ...   
          1994-06-23  0.341667  4.216667  2.341667  2.583333  0.450000  ...   
          1994-06-24  1.183333  3.783333  2.958333  1.458333  1.191667  ...   
...                        ...       ...       ...       ...       ...  ...   
12029071  2014-08-25  1.053333  3.640000  0.220000  0.253333  0.333333  ...   
          2014-08-26  1.040000  3.833333  0.686667  0.233333  0.106667  ...   
          2014-08-27  1.233333  3.346667  0.240000  0.286667  0.433333  ...   
          2014-08-28  1.053333  3.920000  1.273333  0.166667  0.300000  ...   
          2014-08-29  1.173333  3.093333  0.813333  0.340000  0.386667  ...   

                            14        15        16        17        18  \
ProfileID date                                                           
1         1994-06-20  1.433333  2.641667  2.116667  5.541667  5.275000   
          1994-06-21  3.150000  1.825000  4.016667  1.833333  3.466667   
          1994-06-22  1.500000  0.933333  2.791667  6.291667  6.491667   
          1994-06-23  2.666667  4.758333  2.391667  5.233333  3.250000   
          1994-06-24  5.400000  1.041667  4.083333  3.191667  2.216667   
...                        ...       ...       ...       ...       ...   
12029071  2014-08-25  0.200000  0.200000  0.240000  0.533333  1.553333   
          2014-08-26  0.180000  0.200000  0.506667  0.266667  1.820000   
          2014-08-27  0.320000  0.300000  0.246667  0.720000  2.006667   
          2014-08-28  0.473333  0.273333  0.246667  0.246667  1.606667   
          2014-08-29  0.620000  0.386667  0.400000  0.400000  2.200000   

                            19         20         21        22        23  
ProfileID date                                                            
1         1994-06-20  6.533333   4.708333  10.433333  9.508333  4.408333  
          1994-06-21  8.875000  10.850000  12.908333  9.241667  2.625000  
          1994-06-22  4.458333  13.125000  10.750000  6.200000  1.350000  
          1994-06-23  5.958333  10.466667  10.066667  5.016667  0.733333  
          1994-06-24  4.258333   2.725000   0.975000  1.216667  1.608333  
...                        ...        ...        ...       ...       ...  
12029071  2014-08-25  1.120000   3.413333   1.293333  0.346667  0.226667  
          2014-08-26  1.433333   3.640000   1.586667  0.280000  0.286667  
          2014-08-27  1.000000   3.580000   1.466667  0.280000  0.400000  
          2014-08-28  1.333333   3.860000   1.586667  0.293333  0.273333  
          2014-08-29  1.513333   3.893333   1.453333  0.373333  0.206667  

[655818 rows x 24 columns]

### Get households that pass the stictily residential test

In [5]:
mean = df_winter_weekdays.groupby(["ProfileID"]).mean()

# Test 1: Energy between 9-5 
i = np.arange(9,17)
sum_df = pd.DataFrame(index = mean.index, columns = ["9_to_5", 'total'])
sum_df = sum_df.fillna(0)
for x in i:
    temp = mean.iloc[:,x]
    sum_df['9_to_5'] = sum_df['9_to_5'] + temp

sum_df['total'] =  mean.sum(axis = 1)

sum_df["percentage"] = (sum_df["9_to_5"]/sum_df["total"]) * 100

def check_9_to_5(df):
    if df > 50:
        return 'B'
    else: 
        return 'R'

sum_df['Labels'] = sum_df['percentage'].apply(check_9_to_5)

# 173 households labeled as Businesses for the 9-to-5 check
sum_df[sum_df['percentage'] > 50]

,9_to_5,total,percentage,Labels
ProfileID,,,,
66,7.314488,14.168734,51.624145,B
217,29.234047,48.776469,59.934734,B
221,11.493651,22.538165,50.996394,B
232,46.414889,88.394214,52.508967,B
284,22.920155,44.093986,51.980230,B
...,...,...,...,...
12028369,19.029901,37.612317,50.594864,B
12028403,6.608640,8.126843,81.318658,B
12028447,24.028575,45.127989,53.245392,B


In [5]:
# Tests 2: ToP
top_df = pd.DataFrame()

top_df['Time'] = mean.idxmax(axis=1)

def check_top(df):
    if int(df) > 9 and int(df) < 17:
        return 'B'
    else:
        return 'R'  

top_df['Labels'] = top_df['Time'].apply(check_top)

### Extract households that are residential according to test

In [6]:
new_df = pd.DataFrame()

new_df['ToP'], new_df['Sum'] = top_df['Labels'], sum_df['Labels']

data_classes = ['R','B']
d = dict(zip(data_classes, range(0,2)))

new_df['ToP'],new_df['Sum'] = new_df['ToP'].map(d, na_action='ignore'),new_df['Sum'].map(d, na_action='ignore')

new_df['Total'] = new_df.sum(axis = 1)

In [7]:
residential_ids = new_df[new_df['Total'] == 0].index.values # Contains the profile ids that have typical residential behaviour

# Extract Residential households that have complete data
### Housholds must have:
### - Atleast 3 months of winter weekday data, i.e. +-60 days of data

In [8]:
df_winter_weekdays.reset_index(inplace = True)

In [9]:
# Remove all households with less than three winter months
winter_months_ids = []

for i in df_winter_weekdays['ProfileID'].unique():
    if len(df_winter_weekdays[df_winter_weekdays['ProfileID'] == i]) >= 30:

        winter_months_ids.append(i) # Contains the profile Ids that have a complete winter months data

winter_months_ids = np.asarray(winter_months_ids)

In [10]:
profileIDs = np.intersect1d(winter_months_ids, residential_ids)

In [11]:
len(profileIDs)

9993

# Filter out all the ids from the above two extractions steps

In [12]:
cleaned_df = df_winter_weekdays[df_winter_weekdays['ProfileID'].isin(profileIDs)]
cleaned_df.head()

,ProfileID,date,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,1,1994-06-20,1.058333,1.625000,0.341667,2.100000,1.225000,1.058333,4.808333,4.091667,...,1.433333,2.641667,2.116667,5.541667,5.275000,6.533333,4.708333,10.433333,9.508333,4.408333
1,1,1994-06-21,0.425000,2.350000,0.550000,0.291667,2.100000,0.591667,4.066667,2.791667,...,3.150000,1.825000,4.016667,1.833333,3.466667,8.875000,10.850000,12.908333,9.241667,2.625000
2,1,1994-06-22,0.741667,2.325000,0.583333,1.191667,1.225000,0.575000,4.066667,3.350000,...,1.500000,0.933333,2.791667,6.291667,6.491667,4.458333,13.125000,10.750000,6.200000,1.350000
3,1,1994-06-23,1.166667,1.883333,0.450000,0.541667,2.333333,0.341667,4.216667,2.341667,...,2.666667,4.758333,2.391667,5.233333,3.250000,5.958333,10.466667,10.066667,5.016667,0.733333
4,1,1994-06-24,2.408333,0.591667,0.316667,2.341667,0.550000,1.183333,3.783333,2.958333,...,5.400000,1.041667,4.083333,3.191667,2.216667,4.258333,2.725000,0.975000,1.216667,1.608333


# Extract gauss_fit features
### Extract the mean amplitudes
### Sigma values
### Mean amplitude deviation
### Mean Daily consumptions
### Deviation in daily consumption
### Time of peaks


In [13]:
reduced_list = random.choices(profileIDs,k = 20)

In [14]:
len(profileIDs)

9993

In [16]:
# # Create Dataframe
# cols = pd.MultiIndex.from_tuples([("ProfileID",''),
#                                 ('H_offset', 'H_offset'),
#                                  ("Morning", "sigma1"), 
#                                   ("Morning", "sigma2"), 
#                                   ("Morning", "mu1"),
#                                   ("Morning", "A1"),
#                                   ("Afternoon", "sigma3"), 
#                                   ("Afternoon", "sigma4"), 
#                                   ("Afternoon", "mu2"),
#                                   ("Afternoon", "A2")])

cols = ['ProfileID','H_offset','sigma1','sigma2','mu1','A1','sigma3','sigma4','mu2','A2']

# Create dummy variables
H_offset = 0
sigma1 = 0
sigma2 = 0 
mu1 = 0
A1 = 0 
sigma3 = 0 
sigma4 = 0 
mu2 = 0
A2 = 0

data=[['DROP_ROW',H_offset,sigma1,sigma2, mu1, A1, sigma3, sigma4, mu2,A2]]

gauss_df = pd.DataFrame(data, columns=cols)

for id in tqdm(profileIDs):
    H_offset,sigma1, sigma2, mu1, A1, sigma3, sigma4, mu2, A2, check = extractFIT(cleaned_df,id)
    
    if check == False:
        continue

    data=[[id,H_offset,sigma1,sigma2, mu1, A1, sigma3, sigma4, mu2,A2]]
    temp_df = pd.DataFrame(data, columns=cols)
    temp_df.set_index(['ProfileID'])
    gauss_df = gauss_df.append(temp_df)

gauss_df = gauss_df.set_index(['ProfileID'])

# Store Gaussian Fit features
temp = gauss_df.copy()
temp.drop(['DROP_ROW'],axis = 0, inplace = True)
temp.to_csv('FitFeatures_ALL2.csv')

<ipython-input-16-8fec1e1e7247>:30: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/9993 [00:00<?, ?it/s]

In [34]:
# temporary_df = cleaned_df.groupby(["ProfileID"]).mean()

# Extract the amplitude of every day for each profile

In [35]:
reduced_list = random.choices(profileIDs,k = 10)

In [36]:
t = cleaned_df[cleaned_df['ProfileID'] == 5]
t_series = t.iloc[0]
t_series.drop(labels = ['date'], inplace = True)
t_frame = t_series.to_frame()
t_frame = t_frame.T
t_frame.reset_index(inplace = True)
t_frame.drop(['index'], axis = 1, inplace = True)
t_frame.set_index(['ProfileID'], inplace = True)
t_frame
# df = df.drop(labels = ['ProfileID', 'date'])

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\series.py:4443: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
ProfileID,,,,,,,,,,,,,,,,,,,,,
5,2.97775,1.069333,1.363583,2.370667,1.098,2.6325,7.26575,8.111167,11.583917,9.582167,...,9.444333,17.527,13.849,11.341917,9.104583,15.9335,17.159333,6.229917,4.240667,4.9265


In [37]:
# Create cols
cols = ['ProfileID','A1','A2','mu1','mu2']

# Create dummy variables
mu1 = 0
A1 = 0 
mu2 = 0
A2 = 0

data=[['DROP_ROW',A1,A2,mu1,mu2]]

amplitudes_df = pd.DataFrame(data, columns=cols)

for id in profileIDs:
    for index in cleaned_df[cleaned_df['ProfileID'] == id].index:
        A1, A2, mu1, mu2, check = extractToPs(cleaned_df[cleaned_df['ProfileID'] == id].loc[index])
        if check == False:
            continue
        
        data=[[id,A1,A2,mu1,mu2]]
        temp_df = pd.DataFrame(data, columns=cols)
        # temp_df.set_index(['ProfileID'])
        amplitudes_df = amplitudes_df.append(temp_df)
    
amplitudes_df = amplitudes_df.set_index(['ProfileID'])
temp = amplitudes_df.copy()
temp.drop(['DROP_ROW'],axis = 0, inplace = True)
    

# Calculate the amplitude deviations

In [38]:
temp.head()

,A1,A2,mu1,mu2
ProfileID,,,,
1,4.808333,10.433333,6,21
1,4.066667,12.908333,6,21
1,8.750000,13.125000,10,20
1,4.216667,10.466667,6,20
1,3.783333,5.400000,6,14


In [39]:
def standard_deviation(my_list):
    #calculate population standard deviation of list 
    return (sum((x-(sum(my_list) / len(my_list)))**2 for x in my_list) / len(my_list))**0.5

In [40]:
daily_consumption = pd.DataFrame()

daily_consumption['Daily_Consumption'] = cleaned_df.set_index(["ProfileID"]).sum(axis = 1)

std_deviation_df = pd.DataFrame(index = temp.index.unique())

for id in temp.index.unique():
    std_deviation_df.loc[id,'A1_std'] = standard_deviation(temp.loc[id]['A1'])
    std_deviation_df.loc[id,'A2_std'] = standard_deviation(temp.loc[id]['A2'])
    std_deviation_df.loc[id,'mu1_std'] = standard_deviation(temp.loc[id]['mu1'])
    std_deviation_df.loc[id,'mu2_std'] = standard_deviation(temp.loc[id]['mu2'])
    std_deviation_df.loc[id,'DC_std'] = standard_deviation(daily_consumption.loc[id]['Daily_Consumption'])


In [41]:
std_deviation_df

,A1_std,A2_std,mu1_std,mu2_std,DC_std
ProfileID,,,,,
1,1.524691,4.144478,1.572146,2.016839,15.503554
5,3.734773,4.208459,1.066960,2.274852,25.233687
6,2.612709,4.590361,1.149413,1.614234,18.749896
10,1.586904,1.996307,1.533401,1.876822,13.320836
11,1.880715,2.051301,1.431970,1.579903,14.821876
...,...,...,...,...,...
12028746,3.496038,4.944541,2.625675,2.319962,24.696855
12028750,5.085144,2.385060,2.082483,2.367873,20.613692
12028754,0.912373,3.094797,1.625833,1.727152,6.670844


# Combine the features extracted for the synthetic profiles

In [42]:
# std_deviation_df
# temp.columns = temp.columns.droplevel(0)

combined_df = pd.merge(temp,std_deviation_df, left_index = True, right_index = True)
combined_df.to_csv('FitFeaturescombined_ALL.csv')

In [43]:
combined_df

,A1,A2,mu1,mu2,A1_std,A2_std,mu1_std,mu2_std,DC_std
ProfileID,,,,,,,,,
1,4.808333,10.433333,6,21,1.524691,4.144478,1.572146,2.016839,15.503554
1,4.066667,12.908333,6,21,1.524691,4.144478,1.572146,2.016839,15.503554
1,8.750000,13.125000,10,20,1.524691,4.144478,1.572146,2.016839,15.503554
1,4.216667,10.466667,6,20,1.524691,4.144478,1.572146,2.016839,15.503554
1,3.783333,5.400000,6,14,1.524691,4.144478,1.572146,2.016839,15.503554
...,...,...,...,...,...,...,...,...,...
12029049,1.683333,1.533333,10,17,0.838400,1.103542,2.409524,1.253057,4.146102
12029049,0.450000,1.116667,5,17,0.838400,1.103542,2.409524,1.253057,4.146102
12029049,1.466667,1.450000,6,18,0.838400,1.103542,2.409524,1.253057,4.146102


In [44]:
# def extract_all_amplitudes(df,id):

#     A1, A2, mu1, mu2 = extractToPs(df, id)

#     data=[[id, A1, A2, mu1, mu2]]

#     temp_df = pd.DataFrame(data, columns=cols)
#     temp_df.set_index(['ProfileID'])
#     gauss_df = gauss_df.append(temp_df)

In [45]:
# for id in reduced_list:
#     for row in cleaned_df.loc[id]:
#         print(row)


### Winter Weekdays

In [46]:
# path =r'C:\Users\Jason\thesis_project\data\features\FitFeatures\Winter_Weekdays'
# list_files = []
# for root, dirs, files in os.walk(path):
# 	for file in files:
# 		list_files.append(os.path.join(root,file))

# dfs = []
# res = ''
# for name in list_files:
#     temp_df = pd.read_csv(name,header=[0, 1], index_col = 0)
#     dfs.append(temp_df)

# res = pd.concat(dfs, ignore_index=False)  # concatenate list of dataframes

# res

# Get the amplitude deviation

### Winter-weekdays

In [47]:
# X.reset_index(inplace = True)

# df = X.copy()

# # Extract Season
# df['month'] = df.date.dt.month
# df['season'] = df['month'].apply(lambda x: 'winter' if x in [6, 7, 8] else 'summer') 
# df_winter = df[df['season'] == 'winter'] # Create dataframe with all the winter months, excluding weekends


# # Extract Weekdays
# weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']
# df_winter['day_names'] = df_winter.date.dt.day_name()
# df_winter['daytype'] = df_winter.day_names.where(~df_winter.day_names.isin(weekdays), 'weekday')
# df_winter.drop(['day_names'], axis = 1, inplace = True)
# df_winter_weekdays  = df_winter[df_winter['daytype'] == 'weekday'] # Create dataframe with only weekdays
# df_winter_weekdays.drop(['month', 'season','daytype'], axis = 1, inplace = True)
# df_winter_weekdays.set_index(['ProfileID','date'], inplace=True)

# # df_winter_weekdays.to_csv("Winter_Weekdays_Dataframe.csv")


# Extract the amplitude devition

In [48]:
# # Get the profile IDs
# ids = res.index.values

In [49]:
# df_winter_weekdays.reset_index(["date"],inplace = True)

In [50]:
# df_winter_weekdays = df_winter_weekdays[df_winter_weekdays.index.isin(ids)]

In [51]:
# df_winter_weekdays.drop(['date'],axis = 1,inplace = True)

In [52]:
# df_winter_weekdays.reset_index(inplace = True)

In [53]:
# df_mean = df_winter_weekdays.groupby(['ProfileID']).mean()